# SmartException: User Guide

An interactive **exception coach** for IPython/Jupyter notebooks: instead of dumping a raw traceback, it can show a small, actionable **diagnosis card** with hints, and (when multiple heuristics match) let you switch between plausible explanations.

> Tip: This notebook is written to be runnable top-to-bottom, but some cells intentionally trigger errors to demonstrate the UI.


## What this package is for (and what it is not)

**For:**
- Teaching and learning in notebooks: turn common mistakes into actionable hints.
- Research workflows: quickly spot the *likely* cause of an exception and the relevant line.
- Building notebook tools: provide friendlier errors while keeping raw tracebacks available.

**Not for:**
- Security/sandboxing (it renders HTML/Markdown for a trusted local notebook).
- Full static analysis or formal proof of correctness (diagnoses are heuristics).

### Mental model

Think of SmartException as a pipeline:
1. **Activation** installs a custom exception handler in the current IPython session.
2. When an exception happens, the handler extracts a best-effort **source location**.
3. A registry of small **diagnosers** runs; each returns a confidence score and a message.
4. The UI shows the top diagnosis and lets you switch to alternatives; a **Details** panel contains the full traceback.


## Installation & Requirements

SmartException is designed to import even if UI dependencies are missing.

### Minimal requirements
- Python 3.10+

### Optional dependencies

| Optional package | Install hint | Enables |
|---|---|---|
| `IPython` | `pip install ipython` (or via conda) | Jupyter/IPython integration (`activate`, custom handlers) |
| `ipywidgets` | `pip install ipywidgets` | Interactive diagnosis **cards** UI |
| `markdown` | `pip install markdown` | Richer rendering of explanations/hints (Markdown → HTML) |

If `ipywidgets` is missing, SmartException will fall back to printing the raw traceback.


## Quickstart (5 minutes)

This quickstart does three things:
1. Imports SmartException.
2. Activates the exception handler.
3. Demonstrates a diagnosis card (without interrupting the notebook).

We intentionally *catch* the exception, then call `smart_exception_handler` directly so the cell can finish.
In normal use you typically **do not** call the handler yourself: you just activate it and let it run.


In [1]:
import gu_toolkit
gu_toolkit.setup()

🔧 Initializing GU Toolkit...
✓ Matplotlib backend set to 'widget'
[gu_toolkit] Skip (disabled) gu_toolkit.plugins.example
[gu_toolkit] Loaded gu_toolkit.plugins.NamedFunction (exports=1, hook=False)
[gu_toolkit] Loaded gu_toolkit.plugins.SmartException.SmartException (exports=1, hook=True)
[gu_toolkit] Loaded gu_toolkit.plugins.SmartFigure.SmartFigure (exports=9, hook=False)
[gu_toolkit] Loaded gu_toolkit.plugins.SmartParameters.SmartParameters (exports=4, hook=False)
[gu_toolkit] Loaded gu_toolkit.plugins.numpify (exports=1, hook=False)
[gu_toolkit] Exported 951 names into the notebook namespace.
🎓 GU Toolkit Ready.


### Discovering the public surface

SmartException is a small module; you can discover what it exposes in three complementary ways:
- `__all__` (what `from SmartException import *` would export)
- documented “Public entrypoints” in the module docs
- “public” names by convention (not starting with `_`)


In [ ]:
import inspect

all_list = list(getattr(se, "__all__", []))
public_names = sorted([name for name in dir(se) if not name.startswith("_")])

print("In __all__:", all_list)
print()

# A compact "API snapshot": only show functions/classes (skip imported modules, constants, etc.)
api_items = []
for name in public_names:
    obj = getattr(se, name)
    if inspect.isfunction(obj) or inspect.isclass(obj):
        api_items.append((name, type(obj).__name__))

print("Public functions/classes (by convention):")
for name, kind in api_items:
    print(f"- {name} ({kind})")

print()
print("Plugin metadata (if present):")
for key in ["__gu_exports__", "__gu_priority__", "__gu_enabled__"]:
    print(f"- {key} =", getattr(se, key, None))


In [ ]:
# --- Activate SmartException in this IPython session ---
# (If IPython is not available, this is a no-op.)
se.activate(verbose=True)


### A tiny demo (without stopping the notebook)

We will execute code that fails, catch the exception, and then ask SmartException to render a diagnosis card for it.
This is handy for:
- building “error galleries” for course materials,
- writing tests/demos where you don't want the cell to abort.


In [ ]:
from typing import Optional

def render_card_for(code: str, *, global_ns: Optional[dict] = None) -> None:
    """Execute `code` and render the SmartException UI for the resulting exception.

    Parameters
    ----------
    code:
        A Python snippet (single or multi-line).
    global_ns:
        Optional globals to use for exec. If omitted, a fresh dict is used.

    Notes
    -----
    - This function deliberately catches the exception and then calls
      `se.smart_exception_handler(...)` directly.
    - In normal interactive use you typically rely on `se.activate()` and let
      uncaught exceptions be handled automatically.
    """
    ns = {} if global_ns is None else dict(global_ns)
    try:
        exec(code, ns, ns)
        print("No exception raised.")
    except Exception as e:
        print(f"Captured: {type(e).__name__}: {e}")
        se.smart_exception_handler(None, type(e), e, e.__traceback__)


render_card_for("sin(0.5)")


### (Optional) Demo the *automatic* handler

Run the next cell to trigger an **uncaught** exception.
SmartException should render a diagnosis card automatically.

> After that cell errors, continue to the next cell (Jupyter lets you keep running subsequent cells).


In [2]:
sin(0.5)


0.479425538604203

In [3]:
# --- Cleanup (recommended) ---
# If you're using SmartException temporarily in a notebook, turn it off when done.
se.deactivate(verbose=True)


Output()

## Core Concepts

This section introduces the key objects and the workflow.


### 1) Activation lifecycle (`activate` / `deactivate`)

`activate()` installs a custom exception handler into the active IPython session; `deactivate()` restores default handling.

**Gotchas**
- Activation affects the *current IPython session*, not just one cell.
- If you share a notebook, collaborators may have different widget configurations.
- Always keep a path back to raw tracebacks (SmartException provides a “Details” panel).


In [ ]:
se.activate(verbose=True)
print("Now the custom exception handler is installed (in this IPython session).")
se.deactivate(verbose=True)
print("Handler removed; tracebacks are back to default.")


### 2) Diagnosis cards and confidence

Each diagnoser returns a **confidence score** in `[0, 1]` and a small message (title, explanation, hint).
The UI ranks candidates by confidence and lets you switch among them.

**Gotchas**
- Diagnoses are heuristics; treat them as a fast *starting point*, not an oracle.
- Multiple diagnosers can match the same exception; the UI may show several plausible explanations.


In [ ]:
# We'll render a card for a 'power operator' confusion: using ^ instead of **.
se.activate(verbose=False)

render_card_for("""x = 1.5
y = x^2  # <-- Python interprets ^ as bitwise XOR
""")

se.deactivate(verbose=False)


### 3) `GuideError`: a guaranteed, “teachable” error

Raise `GuideError` when you want an exception to *always* show a guidance card with 100% confidence.

This is useful for:
- course notebooks (guardrails for students),
- API wrappers that can detect misuse early,
- cleaner error messages in interactive exploration.

**Gotchas**
- `GuideError` is still an exception; it should be used for genuinely exceptional situations.
- Keep messages short and actionable; include a concrete next step in `hint=`.


In [ ]:
se.activate(verbose=False)

def require_probability_vector(p):
    p = list(p)
    if not p:
        raise se.GuideError(
            "Empty vector: expected probabilities that sum to 1.",
            hint="Provide a non-empty list of nonnegative numbers.",
        )
    if any(x < 0 for x in p):
        raise se.GuideError(
            "Probabilities must be nonnegative.",
            hint="Check for sign mistakes (e.g. subtracting instead of adding).",
        )
    s = sum(p)
    if abs(s - 1.0) > 1e-9:
        raise se.GuideError(
            f"Probabilities must sum to 1 (got {s}).",
            hint="Normalize with: p = [x/s for x in p].",
        )
    return p

# Intentionally misuse it:
render_card_for(
    "require_probability_vector([0.2, 0.2])",
    global_ns={"require_probability_vector": require_probability_vector, "se": se},
)

se.deactivate(verbose=False)


### 4) Extending SmartException with custom diagnosers

To add a new heuristic, subclass `Diagnosis` and register it with `register_diagnosis`.
A diagnoser must be:
- deterministic,
- fast,
- side-effect free (beyond internal cached match state).

**Gotchas**
- The diagnoser registry is global within the module; re-running the same registration cell can create duplicates.
  A common workflow is: define custom diagnosers once per kernel session.


In [ ]:
# Idempotent helper: avoids repeated registration when re-running this notebook cell.
def register_diagnosis_once(module, cls):
    registry = getattr(module, "_DIAGNOSERS", None)
    if isinstance(registry, list) and any(type(d).__name__ == cls.__name__ for d in registry):
        return cls
    return module.register_diagnosis(cls)


class DivisionByZeroDiagnosis(se.Diagnosis):
    """Teach common numeric causes of ZeroDivisionError in notebooks."""

    def _check_condition(self, ctx: "se.ExceptionContext") -> float:  # type: ignore[name-defined]
        return 0.95 if isinstance(ctx.evalue, ZeroDivisionError) else 0.0

    def _generate_info(self, ctx: "se.ExceptionContext") -> tuple[str, str, str]:  # type: ignore[name-defined]
        return (
            "Division by Zero",
            "You divided by zero. In numerical work this often happens because a denominator hit 0 unexpectedly.",
            "Print or assert the denominator before dividing (e.g. `assert den != 0`).",
        )


register_diagnosis_once(se, DivisionByZeroDiagnosis)

se.activate(verbose=False)
render_card_for("1 / 0")
se.deactivate(verbose=False)


## Common Tasks / Recipes

Each recipe is a small, searchable card: **Task → minimal code → how to verify**.


### Recipe: Turn SmartException on for a teaching session

**Task:** activate at the top of a notebook.

**Verify:** trigger a small error and check that you get a card (and that Details shows a traceback).


In [ ]:
se.activate(verbose=True)
print("Try running: 1/0  (or any other error) to verify the card renders.")


### Recipe: Turn SmartException off (restore standard tracebacks)

**Task:** deactivate when you no longer want cards.

**Verify:** run an error and confirm you see the usual traceback formatting.


In [ ]:
se.deactivate(verbose=True)
print("Now errors should show the default traceback again.")


### Recipe: Build a small “error gallery” for students

**Task:** show several common mistakes and their diagnosis cards without aborting the notebook.

**Verify:** each snippet should render a card.


In [ ]:
se.activate(verbose=False)

snippets = {
    "Implicit multiplication (2x)": "2x + 1",
    "Using ^ for powers": "x = 2.0\nx^2",
    "Missing prefix (sin)": "sin(0.3)",
}

for name, code in snippets.items():
    print("\n" + "=" * 80)
    print(name)
    render_card_for(code)

se.deactivate(verbose=False)


### Recipe: Provide course-specific guardrails with `GuideError`

**Task:** raise a `GuideError` with an actionable hint.

**Verify:** the diagnosis card should show with “Match: 100%”.


In [ ]:
se.activate(verbose=False)

def require_even(n: int) -> int:
    if n % 2 != 0:
        raise se.GuideError(
            f"Expected an even integer, got {n}.",
            hint="Try: n = 2*k for some integer k. If you computed n, print intermediate values.",
        )
    return n

render_card_for("require_even(7)", global_ns={"require_even": require_even, "se": se})

se.deactivate(verbose=False)


## Applications (mini-projects)

These are small, reproducible projects designed for researchers and educators.


### Application 1: A math-syntax “coach” in a Real Analysis notebook

**Motivating question:** How can we give students immediate feedback on common notebook mistakes (like `2x` or `^`)?

We'll run a few representative mistakes and inspect the diagnosis cards.


In [6]:
# Implicit multiplication
f = 2x + 1

Output()

In [9]:
# Power operator confusion
y = 1.5^2
    

Output()

**What you learned / how to adapt:**

- You can embed an “error gallery” early in the term.
- Instructors can add course-specific diagnosers (see Application 2).
- Students still have access to the full traceback via the Details panel.


In [ ]:
# Missing prefix for common functions


In [10]:
linspace(0,1,10)

Output()

### Application 2: Add a custom diagnosis for numerical experiments

**Motivating question:** In numerical labs, division-by-zero often means a denominator hit 0 due to a parameter choice.

We'll add a custom diagnoser and then re-run a failing snippet.


In [ ]:
# The custom diagnoser was registered earlier (DivisionByZeroDiagnosis).
# We'll demonstrate it in a more "numerical experiment" flavored snippet.

se.activate(verbose=False)

render_card_for("""import math
x = 0.0
y = 1.0 / x
""")

se.deactivate(verbose=False)


**What you learned / how to adapt:**

- Custom diagnosers let you encode “lab lore” directly into the notebook environment.
- Keep the hint actionable: what should the student print, assert, or plot next?


### Application 3: Enforce invariants in a small workflow with `GuideError`

**Motivating question:** When prototyping, you often want to fail fast with a clear message if invariants break.

We'll implement a tiny, deterministic “mini-project”: normalizing a discrete distribution and validating it.


In [ ]:
se.activate(verbose=False)

def normalize(p):
    p = [float(x) for x in p]
    if not p:
        raise se.GuideError("Expected a non-empty list.", hint="Provide at least one weight.")
    if any(x < 0 for x in p):
        raise se.GuideError(
            "Weights must be nonnegative.",
            hint="Check your formula; negative weights are usually a bug.",
        )
    s = sum(p)
    if s == 0:
        raise se.GuideError("All weights are zero.", hint="At least one entry must be positive.")
    return [x / s for x in p]

def entropy(p):
    p = normalize(p)
    import math
    return -sum(x * math.log(x) for x in p)

render_card_for("entropy([0.0, 0.0, 0.0])", global_ns={"entropy": entropy, "se": se})

se.deactivate(verbose=False)


**What you learned / how to adapt:**

- `GuideError` is a clean way to make invariants explicit.
- You can treat these messages as part of your *teaching interface* for notebooks.


## Advanced Guide

This section is for power users, maintainers, and collaborators.


### Customization & configuration

- `activate(verbose=True)` prints log messages about activation.
- The card appearance (colors/icons) is determined by confidence thresholds.
- Markdown rendering is optional; without `markdown`, text is displayed as-is.

If you want deep UI customization (themes, extra buttons), look for the UI renderer section in the source.


In [ ]:
# Quick check: are optional deps available in *your* environment?
def check_optional():
    mods = {}
    for name in ["IPython", "ipywidgets", "markdown"]:
        try:
            __import__(name)
            mods[name] = True
        except Exception:
            mods[name] = False
    return mods

check_optional()


### Extending the package

**Extension point:** new diagnosers via `Diagnosis` + `register_diagnosis`.

Invariants your extensions should preserve:
- Never raise from `diagnose` (SmartException is defensive, but you should be too).
- Be deterministic: avoid randomness, timestamps, I/O.
- Return confidence in `[0, 1]` and keep the explanation short.

Practical advice:
- Start with *narrow* checks that are often correct (high confidence), then broaden.
- Prefer a clear hint (“Try X”) over a long explanation.


### Performance & scaling

SmartException is designed to be lightweight:
- Diagnosers are simple heuristics.
- It only runs when exceptions occur.

If you add many custom diagnosers:
- keep each `_check_condition` fast (string checks / small regexes),
- avoid expensive imports inside diagnosers,
- profile by timing the code that triggers and renders the exception.


In [ ]:
# A tiny, high-level timing pattern (not a benchmark, just a sanity check).
import time

se.activate(verbose=False)

t0 = time.perf_counter()
render_card_for("1/0")
t1 = time.perf_counter()

se.deactivate(verbose=False)

print(f"Render time (one card): {t1 - t0:.4f} seconds")


### Interoperability

- **NumPy/SciPy**: SmartException doesn't replace numerical error handling, but it can surface common mistakes faster.
- **Plotting**: cards are independent of plotting backends.
- **Other notebook tools**: because activation uses IPython's custom exception hooks, other tools that modify exception handling may interact.


### Troubleshooting

**I only see raw tracebacks, no card**
- Ensure you're running in IPython/Jupyter.
- Ensure `ipywidgets` is installed and enabled in your environment.
- Try calling `render_card_for(...)` (manual handler invocation) to isolate activation issues.

**Cards render but don't show Markdown formatting**
- Install `markdown` to enable Markdown→HTML conversion.

**Location/caret points somewhere surprising**
- For runtime exceptions, location is best-effort and depends on traceback frames.
- For code run via `exec`/`eval`, line mapping can be limited.

**How to provide a minimal reproducible example (MRE)**
- Include: Python version, notebook environment, a minimal snippet that triggers the exception.
- Mention whether you activated SmartException and whether widgets are enabled.


### Contributing

If you're extending SmartException itself:
- Add built-in diagnosers near the built-in diagnosers section in the module.
- Keep each diagnoser self-contained and testable.
- Add tests in a small unit test file (e.g. `tests/test_smart_exception.py`) or in a dedicated notebook-based TestSuite.
- When diagnosing new error patterns, prefer high precision first; add lower-confidence alternatives only when useful.


## API Reference (lightweight)

This is a curated index. For authoritative parameter details, consult docstrings in the source.

### Most important entrypoints
- `activate(verbose=False)` — install the handler in the current IPython session.
- `deactivate(verbose=False)` — restore default exception handling.
- `smart_exception_handler(shell, etype, evalue, tb, tb_offset=None)` — render ranked diagnosis cards.
- `GuideError(message, hint='')` — raise to show a 100% confidence guidance card.

### Extension API
- `Diagnosis` — base class for new heuristics.
- `register_diagnosis(cls)` — register a `Diagnosis` subclass.

### Data structures (useful for maintainers)
- `SourceLocation` — code line, caret, line number.
- `DiagnosisCard` — what gets rendered.
- `ExceptionContext` — what diagnosers receive.


## Next Steps

Suggested reading order:
1. Quickstart
2. Core Concepts (Activation + GuideError)
3. Common Tasks / Recipes
4. Applications (especially if you teach)
5. Advanced Guide (if you plan to extend or contribute)

Validation:
- If your project includes a notebook-based TestSuite, run it after installation.
- If you maintain SmartException, keep a small set of “error gallery” snippets to prevent regressions.
